<a href="https://colab.research.google.com/github/Deepika1454/Audio-Classification-with-Pre-trained-Model/blob/main/ENTITY_TYPE_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy pymupdf python-docx gradio
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires

In [2]:
import fitz  # PyMuPDF for PDFs
from docx import Document

def extract_text(file_path):
    """Extracts text from PDF, TXT, or DOCX files."""
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".txt"):
        return extract_text_from_txt(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    else:
        return "❌ Unsupported file format! Please upload a PDF, TXT, or DOCX."

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text.strip()

def extract_text_from_txt(txt_path):
    """Extract text from a TXT file."""
    with open(txt_path, "r", encoding="utf-8") as file:
        return file.read().strip()

def extract_text_from_docx(docx_path):
    """Extract text from a DOCX file."""
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs]).strip()


In [3]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def perform_ner(file_path, entity_type):
    """Extract text, perform NER, and filter by selected entity type."""
    text = extract_text(file_path)
    if "❌" in text:
        return text  # Error message

    doc = nlp(text)
    filtered_entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == entity_type]

    if filtered_entities:
        result = "\n🔹 **Named Entities Found:**\n"
        for entity, label in filtered_entities:
            result += f"- {entity} → **{label}**\n"
        return result
    else:
        return f"❌ No entities of type **{entity_type}** found."


In [5]:
import spacy
import gradio as gr
import fitz  # PyMuPDF for PDFs
from docx import Document

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to extract text from different file formats
def extract_text(file_path):
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".txt"):
        return extract_text_from_txt(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    else:
        return "❌ Unsupported file format! Please upload a PDF, TXT, or DOCX."

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text.strip()

def extract_text_from_txt(txt_path):
    with open(txt_path, "r", encoding="utf-8") as file:
        return file.read().strip()

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs]).strip()

# Function to perform NER and filter by selected entity type
def perform_ner(file_path, entity_type):
    text = extract_text(file_path)
    if "❌" in text:
        return text  # Return error message if unsupported file

    doc = nlp(text)
    filtered_entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == entity_type]

    if filtered_entities:
        result = "\n🔹 **Named Entities Found:**\n"
        for entity, label in filtered_entities:
            result += f"- {entity} → **{label}**\n"
        return result
    else:
        return f"❌ No entities of type **{entity_type}** found."

# List of entity types for selection
entity_types = ["PERSON", "ORG", "GPE", "DATE", "PRODUCT", "EVENT", "WORK_OF_ART"]

# Gradio Interface
gr.Interface(
    fn=perform_ner,
    inputs=[gr.File(type="filepath"), gr.Dropdown(choices=entity_types, label="Select Entity Type")],
    outputs="text",
    title="📄 AI-Based Named Entity Recognition (NER) System",
    description="Upload a PDF, TXT, or DOCX file, select an entity type, and see results instantly!",
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f924903adf0c59dc8e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
